In [ ]:
from typing import Any

import numpy
import polars
from datasets import Dataset, load_dataset
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler

In [ ]:
dataset = load_dataset(
	"csv", data_dir="../data", data_files={"train": "train.csv", "test": "test.csv"}, keep_in_memory=True, num_proc=8
)

In [ ]:
dataset = dataset.map(
	lambda samples: {
		"play years_0": [1 if sample == 0 else 0 for sample in samples],
		"play years_1": [1 if sample == 1 else 0 for sample in samples],
		"play years_2": [1 if sample == 2 else 0 for sample in samples],
	},
	input_columns=["play years"],
	remove_columns=["play years"],
	batched=True,
	num_proc=8,
)

In [ ]:
dataset = dataset.map(
	lambda samples: {
		"level_0": [1 if sample == 0 else 0 for sample in samples],
		"level_1": [1 if sample == 1 else 0 for sample in samples],
		"level_2": [1 if sample == 2 else 0 for sample in samples],
	},
	input_columns=["level"],
	remove_columns=["level"],
	batched=True,
	num_proc=8,
)

In [ ]:
index_columns: list = ["data_ID", "player_ID"]
label_columns: list = [
	"gender",
	"hold racket handed",
	"play years_0",
	"play years_1",
	"play years_2",
	"level_0",
	"level_1",
	"level_2",
]
data_columns: list = list(set(dataset["train"].column_names).difference(set(index_columns + label_columns)))

In [ ]:
data_columns

In [ ]:
scaler = StandardScaler()

In [ ]:
train_dataset: polars.DataFrame = dataset["train"].to_polars()
train_data: numpy.ndarray = scaler.fit_transform(train_dataset[data_columns].to_numpy())
train_data, validation_data, t_label, v_label = train_test_split(
	train_data, train_dataset[label_columns].to_numpy(), test_size=0.1, random_state=37
)
train_label: dict[str, numpy.ndarray] = {label: t_label[:, i] for i, label in enumerate(label_columns)}
validation_label: dict[str, numpy.ndarray] = {label: v_label[:, i] for i, label in enumerate(label_columns)}

In [ ]:
hyperparameter_grid: dict[str, list[Any]] = {
	"hidden_layer_sizes": [(50,), (100,), (200,), (50, 50), (100, 50), (100, 100)],
	"activation": ["identity", "logistic", "tanh", "relu"],
	"solver": ["adam", "sgd", "lbfgs"],
	"learning_rate": ["constant", "adaptive", "invscaling"],
	"learning_rate_init": [0.001, 0.01, 0.1],
	"max_iter": [200, 300, 400, 500],
	"alpha": [0.0001, 0.001, 0.01],
	"batch_size": ["auto", 32, 64, 128],
}

In [ ]:
best_classifiers: dict = {}
for label in label_columns:
	print(f"Tune classifier for {label}")
	classifier = MLPClassifier(random_state=37)

	tuner = RandomizedSearchCV(
		classifier, hyperparameter_grid, n_iter=1000, cv=5, scoring="roc_auc_ovr", n_jobs=-1, verbose=1
	)

	tuner.fit(train_data, train_label[label])

	best_classifiers[label] = tuner.best_estimator_

	y_pred_proba = best_classifiers[label].predict_proba(validation_data)[:, 1]
	auc_score = roc_auc_score(validation_label[label], y_pred_proba, multi_class="ovr")
	print(f"AUC score of {label} classifier: {auc_score:.4f}")

In [ ]:
test_data: polars.DataFrame = scaler.transform(dataset["test"].select_columns(data_columns).to_polars().to_numpy())

In [ ]:
predictions: dict = {"data_ID": dataset["test"]["data_ID"]}
for label in label_columns:
	predictions[label] = best_classifiers[label].predict(test_data).tolist()

In [ ]:
test_result = Dataset.from_dict(predictions)

In [ ]:
test_result

In [ ]:
test_result.to_csv("../dist/result_mp.csv", num_proc=8)